In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
from talos.metrics.keras_metrics import fmeasure
from keras.models import load_model

#----------------------------------------------------------
data = pd.read_csv('conventional_data_unique.csv', index_col='CID')
y_ = data['consensus_act']

X_all = data.drop(['consensus_act', 'gmin'], axis = 1)
X_all.replace([np.inf, -np.inf], np.nan, inplace=True) #replace all infinite values with NaN

X_all = X_all.dropna(axis=1, how='all') #Drop any column that has all NaN
X_all.fillna(value = X_all.mean(axis=0), inplace=True) #Impute mean in place on NaN

#X_all.loc[:, X_all.isnull().any()] See which column still had NaN after prepocessing. gmin was removed
print('Shape of X_all before removing low variance features:', X_all.shape)

#0.16 =(.8 * (1 - .8)) that is the threshold for features that are constant in 80% of the instances
feat = VarianceThreshold(threshold=0.16)
feat.fit_transform(X_all)
X_all = X_all[X_all.columns[feat.get_support(indices=True)]] #retain column names
print('Shape of X_all after removing low variance features:', X_all.shape)


#----------------------------------------------------------------------
n_splits=5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=23)
cvscores=[]
_model = load_model('conv_feat_unique_model_val.h5', custom_objects={'fmeasure': fmeasure})

Predictions_DF = pd.DataFrame()
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_all, y_)):
    print("Testing on fold " + str(fold+1) + " of " + str(n_splits))

    xtrain, xval = X_all.iloc[train_idx], X_all.iloc[val_idx]
    ytrain, yval = y_.values[train_idx], y_.values[val_idx] #Dropped indexes 
    
    ytrain = pd.get_dummies(ytrain)
    yval = pd.get_dummies(yval)
    
    sc = StandardScaler().fit(xtrain)
    xtrain = pd.DataFrame(sc.transform(xtrain), columns = X_all.columns, index=xtrain.index)
    xval = pd.DataFrame(sc.transform(xval), columns = X_all.columns, index=xval.index)
    
    #_model = None
    #_model = load_model('docking_feat_full_model_v2.h5', custom_objects={'fmeasure': fmeasure})

    scores = _model.evaluate(xval, yval, verbose=0)
    print("%s: %.2f%%" % (_model.metrics_names[1], scores[1]))
    cvscores.append(scores[1])
    
    val_class = np.argmax(yval.values, axis=1)
    val_pred = _model.predict_classes(xval)
    #print(confusion_matrix(val_class, val_pred))
    
    #--------------------------------------------------------------------------
    #labels = ['active_agonist', 'active_antagonist', 'inactive', 'inconclusive']
    print(confusion_matrix(val_class, val_pred))
    print(classification_report(val_class, val_pred))
    print ("Precision:", metrics.precision_score(val_class, val_pred, average="macro"))
    print ("Recall:", metrics.recall_score(val_class, val_pred, average="macro"))
    print ("f1_score:", metrics.f1_score(val_class, val_pred, average="macro"))
    f1_macro_all_features = metrics.f1_score(val_class, val_pred, average="macro")
    #------------------------------------------------
    y_proba = _model.predict_proba(xval)
    print('AUROC:', metrics.roc_auc_score(yval, y_proba, average="macro"))
    print('AUPRC:', metrics.average_precision_score(yval, y_proba, average="macro"))
    #-----------------------------------------------------------------------------
    
    #-------------------------------------------------------------------
    #Feature Relevance
    #How is the metric affected by eliminating a feature (LOO)
    features_table = []
    all_features_f1 = f1_macro_all_features

    for col in xval.columns:
        X_modify_test = xval.copy()
        X_modify_test[col] = 0
        y_predi = _model.predict_classes(X_modify_test)
        y_test_ = np.argmax(yval.values, axis=1)
        f1 = metrics.f1_score(y_test_, y_predi, average="macro")

        aa = []
        aa.append(col)
        bb = all_features_f1 - f1
        aa.append(bb)
        aa.append(f1) 
        features_table.append(aa)

    feat_DF2 = pd.DataFrame(features_table, columns = ['Features', 'f1_delta', 'f1'])
    feat_DF_ = feat_DF2.sort_values('f1_delta', ascending = False)
    feat_DF_.head()
    feat_DF_.to_csv('Unique_feat_DL_Run_' + str(fold+1) + '.csv')
    #----------------------------------------------------------------------------
    #Get Predictions
    compare = pd.DataFrame(np.hstack((val_class.reshape(val_class.shape[0],1),
                              val_pred.reshape(val_pred.shape[0],1))),
                              columns = ['consensus_act', 'predicted_act'], index=xval.index)

    compare['score'] = np.where(compare['consensus_act'] == compare['predicted_act'], 'correct', 'incorrect')
    #compare['score'] = compare.apply(lambda x: 'correct' if x['consensus_act'] == x['predicted_act'] else 'incorrect', axis=1)
    compare.to_csv('Unique_DL_Predictions_Run'+str(fold+1) + '.csv')
    
    Predictions_DF = Predictions_DF.append(compare)

Predictions_DF.to_csv('Unique_DL_Predictions.csv')
#---------------------------------------------------------------------------
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))  
print(cvscores)

Using TensorFlow backend.


Shape of X_all before removing low variance features: (7665, 17967)
Shape of X_all after removing low variance features: (7665, 2544)
Testing on fold 1 of 5
fmeasure: 0.91%
[[ 27   0   1   0]
 [  1  11   0   5]
 [  0   1 954  39]
 [  5   2  86 403]]
             precision    recall  f1-score   support

          0       0.82      0.96      0.89        28
          1       0.79      0.65      0.71        17
          2       0.92      0.96      0.94       994
          3       0.90      0.81      0.85       496

avg / total       0.91      0.91      0.91      1535

Precision: 0.855472153098
Recall: 0.845900772281
f1_score: 0.84680861014
AUROC: 0.968570135553
AUPRC: 0.889869280581
Testing on fold 2 of 5
fmeasure: 0.91%
[[ 25   0   2   1]
 [  1  10   0   6]
 [  1   1 953  39]
 [  6   4  73 412]]
             precision    recall  f1-score   support

          0       0.76      0.89      0.82        28
          1       0.67      0.59      0.62        17
          2       0.93      0.96    

In [4]:
Predictions_DF.shape

(7665, 3)

In [5]:
Predictions_DF.head()

,consensus_act,predicted_act,score
CID,,,
177,3,2,incorrect
178,2,2,correct
227,2,2,correct
240,2,2,correct
244,2,2,correct


In [7]:
xtrain.shape

(6133, 2544)

In [6]:
confusion_matrix(Predictions_DF['consensus_act'], Predictions_DF['predicted_act'])

array([[ 127,    0,    8,    5],
       [   5,   46,    2,   29],
       [   1,    3, 4750,  213],
       [  15,   10,  406, 2045]])